In [1]:
import pandas as pd
import itertools
import skill_transform
import time

# Dice data

# Dictionany skill

In [2]:
start_time = time.time()

import glob
allFiles = glob.glob("dice-full*.json")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_json(file_)
    list_.append(df)
skill_dice_jd = pd.concat(list_)

dict_skill = skill_dice_jd.skills.map(lambda t: t.split(','))

dict_skill = dict_skill.map(lambda t: [ skill_transform.skill_transform(x) for x in t ])
dict_skill = dict_skill.map(lambda t: [ x for x in t if len(x) > 0 and len(x) < 30 ])

import cPickle as pickle
musthave_naruki_data = pickle.load( open( "musthave_dice_naruki_data.p", "rb" ) )
dict_skill_filter = filter(lambda t: len(t) > 0, dict_skill)

train_data = musthave_naruki_data + dict_skill_filter

def filter_list(L):
    return filter(lambda t: len(t) > 1, L)

train_data = [ filter_list(d) for d in train_data ]
train_data = filter_list(train_data)

end_time = time.time()
print end_time - start_time

print len(train_data)

1044.15242505
1448762


In [32]:
musthave_naruki_data = [ [skill_transform.skill_transform(x) for x in l] for l in musthave_naruki_data]

In [42]:
len(train_data)

1439549

In [3]:
import cPickle as pickle
dice_data_train = train_data
pickle.dump( dice_data_train, open( "dice_data_train.p", "wb" ) )

In [81]:
from collections import Counter
dice_data_train = pickle.load( open( "dice_data_train.p", "rb" ) )
unique_skill = list(itertools.chain(*train_data))
skill_counter = Counter(unique_skill)

In [90]:
import operator
skill_counter_sorted = sorted(skill_counter.items(), key=operator.itemgetter(1), reverse=True)

In [91]:
skill_counter_sorted

[(u'management', 412890),
 (u'java', 254144),
 (u'sql', 237821),
 (u'security', 236207),
 (u'analysis', 208463),
 (u'test', 207411),
 (u'javascript', 203566),
 (u'netw', 184498),
 (u'agile', 152833),
 (u'manager', 152080),
 (u'linux', 151005),
 (u'html', 138370),
 (u'architecture', 128047),
 (u'hardware', 123309),
 (u'windows', 122368),
 (u'http', 120786),
 (u'python', 112435),
 (u'database', 109524),
 (u'acle', 108763),
 (u'c#', 100257),
 (u'research', 95658),
 (u'net', 93279),
 (u'analyst', 90428),
 (u'css', 89621),
 (u'excel', 86652),
 (u'unix', 84768),
 (u'sql_server', 82302),
 (u'software', 81035),
 (u'angular', 74596),
 (u'mysql', 70275),
 (u'xml', 67918),
 (u'jquery', 67630),
 (u'aspnet', 59525),
 (u'simulation', 58559),
 (u'vmware', 54473),
 (u'qa', 53459),
 (u'lifecycle', 53387),
 (u'sales', 52826),
 (u'script', 52565),
 (u'business_requirements', 51597),
 (u'cisco', 50351),
 (u'j2ee', 49483),
 (u'model', 49089),
 (u'scrum', 48027),
 (u'powerpoint', 47462),
 (u'sdlc', 46700),


In [6]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   

In [33]:
import multiprocessing
from gensim.models import word2vec
start_time = time.time()

dice_model_300d = word2vec.Word2Vec(musthave_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=1, 
          window=20, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

dice_model_300d.init_sims(replace=True)
end_time = time.time()
print end_time - start_time

2017-06-13 15:37:45,894 : INFO : collecting all words and their counts
2017-06-13 15:37:45,895 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-13 15:37:46,022 : INFO : PROGRESS: at sentence #10000, processed 216479 words, keeping 10763 word types
2017-06-13 15:37:46,074 : INFO : PROGRESS: at sentence #20000, processed 316511 words, keeping 19445 word types
2017-06-13 15:37:46,132 : INFO : PROGRESS: at sentence #30000, processed 403006 words, keeping 27286 word types
2017-06-13 15:37:46,185 : INFO : PROGRESS: at sentence #40000, processed 489603 words, keeping 34013 word types
2017-06-13 15:37:46,240 : INFO : PROGRESS: at sentence #50000, processed 579752 words, keeping 40572 word types
2017-06-13 15:37:46,290 : INFO : PROGRESS: at sentence #60000, processed 666564 words, keeping 45530 word types
2017-06-13 15:37:46,332 : INFO : PROGRESS: at sentence #70000, processed 751300 words, keeping 51355 word types
2017-06-13 15:37:46,379 : INFO : PROGRESS: at 

26.5635719299


In [8]:
dice_model_300d.wv.save_word2vec_format("dice_model_300d.model")

2017-06-13 13:04:36,898 : INFO : storing 111159x300 projection weights into dice_model_300d.model


In [80]:
dice_model_300d.similar_by_word(skill_transform.skill_transform("hive"))

[(u'pig', 0.9596650004386902),
 (u'hdfs', 0.9570858478546143),
 (u'hadoop', 0.9506798982620239),
 (u'spark', 0.9487038254737854),
 (u'impala', 0.9463878870010376),
 (u'oozie', 0.9456303119659424),
 (u'flume', 0.941888689994812),
 (u'scoop', 0.9417264461517334),
 (u'sqoop', 0.9416866302490234),
 (u'hbase', 0.9415104389190674)]

In [94]:
get_top('javascript', 20)

[{'sim': 1.0, 'skill': u'javascript'},
 {'sim': 0.6824187, 'skill': u'planisware'},
 {'sim': 0.67997873, 'skill': u'aveksadevelopment'},
 {'sim': 0.67996514, 'skill': u'adroid'},
 {'sim': 0.67994976, 'skill': u'html5+css3'},
 {'sim': 0.67990613, 'skill': u'acle_they'},
 {'sim': 0.67990613, 'skill': u'must_have_solid_c#_pl'},
 {'sim': 0.67987704, 'skill': u'reactjsphp'},
 {'sim': 0.67987704, 'skill': u'apriso_mes'},
 {'sim': 0.67983317, 'skill': u'software_engineerfrontend'},
 {'sim': 0.67977452, 'skill': u'fullstack_c#'},
 {'sim': 0.67965657, 'skill': u'modern_mvc'},
 {'sim': 0.67956746, 'skill': u'such_as_java'},
 {'sim': 0.67956161, 'skill': u'rest_service_node'},
 {'sim': 0.67955256, 'skill': u'cawa'},
 {'sim': 0.67955256, 'skill': u'tool_computer_associate_w'},
 {'sim': 0.67955256, 'skill': u'cawa_scheduler'},
 {'sim': 0.6795525, 'skill': u'html5_css3_jquery_angular'},
 {'sim': 0.6792205, 'skill': u'other_web_ability_to_h'},
 {'sim': 0.6792205, 'skill': u'automate_software_test_fra

In [82]:
df.to_csv("skill_dict_full.csv", encoding='utf-8')

# LSA

In [10]:
import time
import gensim
from gensim import corpora, models, similarities

dictionary = gensim.corpora.Dictionary.load('dice_data_train.dict')
dice_data_train_number = gensim.corpora.MmCorpus("dice_data_train_number.mm")
# tfidf = models.TfidfModel(dice_data_train_number)
# corpus_tfidf = tfidf[dice_data_train_number]
# matrix_tfidf_csc = gensim.matutils.corpus2csc(corpus_tfidf, num_terms=len(dictionary))

2017-06-13 13:07:36,037 : INFO : loading Dictionary object from dice_data_train.dict
2017-06-13 13:07:36,218 : INFO : loaded dice_data_train.dict
2017-06-13 13:07:36,472 : INFO : loaded corpus index from dice_data_train_number.mm.index
2017-06-13 13:07:36,473 : INFO : initializing corpus reader from dice_data_train_number.mm
2017-06-13 13:07:36,475 : INFO : accepted corpus with 1448762 documents, 111159 features, 12531109 non-zero entries


In [11]:
lsa_sim_not_tfidf = similarities.MatrixSimilarity(dice_data_train_number)
def get_top(skill, n = 10):
    sim_list = lsa_sim_not_tfidf[get_lsa_vector(skill)]
    top_index = sim_list.argsort()[-n:][::-1]
    res = []
    for i in top_index:
        res.append({ 'skill': id2skill(i), 'sim': lsa_sim_not_tfidf[i] })
        
    return res

2017-06-13 13:07:36,485 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2017-06-13 13:08:27,247 : INFO : creating matrix with 1448762 documents and 111159 features


MemoryError: 

In [95]:
S

array([   34.96093174,    34.97171159,    34.98943662,    35.03321733,
          35.06705121,    35.07435837,    35.1239101 ,    35.13491926,
          35.1720123 ,    35.18962579,    35.20117388,    35.25574537,
          35.26226862,    35.28171639,    35.30426549,    35.32586727,
          35.36272388,    35.37848797,    35.39524392,    35.41952466,
          35.43036965,    35.44750261,    35.46076201,    35.48109952,
          35.5098468 ,    35.52155672,    35.55784706,    35.60776714,
          35.62944532,    35.63638664,    35.65493304,    35.67940922,
          35.71409718,    35.73728387,    35.77709698,    35.79347836,
          35.82144507,    35.87900603,    35.89684438,    35.93343963,
          35.94837041,    36.00277592,    36.02174947,    36.03611976,
          36.03943334,    36.05589627,    36.06895541,    36.12736839,
          36.13744555,    36.15613426,    36.17428268,    36.19410229,
          36.22063303,    36.23444692,    36.26760551,    36.29391249,
      

In [12]:
import time
import scipy
k=1000

matrix_csc = gensim.matutils.corpus2csc(dice_data_train_number, num_terms=len(dictionary))
start_time = time.time()
U, S, Vt = scipy.sparse.linalg.svds(matrix_csc, k=k, return_singular_vectors='u')
print "k = ", k, "SVD Time:", time.time() - start_time

from scipy import linalg
SS = linalg.diagsvd(S, k, k)
lsa_vector = U.dot(SS)



k =  1000 SVD Time: 873.551267147


In [13]:
def get_lsa_vector(t):
    try:
        return lsa_vector[dictionary.token2id[skill_transform.skill_transform(t)]]
    except:
        return None
def id2skill(id):
    return dictionary[id]

In [14]:
lsa_copus = gensim.matutils.Dense2Corpus(lsa_vector.T)
lsa_sim = similarities.MatrixSimilarity(lsa_copus)

2017-06-13 14:37:38,970 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2017-06-13 14:38:03,133 : INFO : creating matrix with 111159 documents and 1000 features


In [15]:
def get_top(skill, n = 10):
    sim_list = lsa_sim[get_lsa_vector(skill)]
    top_index = sim_list.argsort()[-n:][::-1]
    res = []
    for i in top_index:
        res.append({ 'skill': id2skill(i), 'sim': sim_list[i] })
        
    return res

get_top('react.js')

[{'sim': 1.0, 'skill': u'react'},
 {'sim': 0.841223, 'skill': u'typescript_net'},
 {'sim': 0.84117341, 'skill': u'java_restful_webservice'},
 {'sim': 0.84107405, 'skill': u'developers_node'},
 {'sim': 0.84004641, 'skill': u'flux_strongly_need'},
 {'sim': 0.82230437, 'skill': u'react_native__javascript'},
 {'sim': 0.78745151, 'skill': u'native_swift'},
 {'sim': 0.76682037, 'skill': u'some_design_skills'},
 {'sim': 0.76359987, 'skill': u'expert_javascript_skillsnode'},
 {'sim': 0.72596657, 'skill': u'redux_interfacing'}]

In [38]:
get_top('pandas', 10)

TypeError: 'NoneType' object is not iterable

In [50]:
get_top('SQL')

[{'sim': 1.0, 'skill': u'sql'},
 {'sim': 0.76782548, 'skill': u'manual_tester__vb_script'},
 {'sim': 0.76780689, 'skill': u'agilemanagement'},
 {'sim': 0.76780689, 'skill': u'buildtool'},
 {'sim': 0.76778829, 'skill': u'dynamcs_nav'},
 {'sim': 0.76778448, 'skill': u'sql_tool'},
 {'sim': 0.76776969, 'skill': u'48_years_strong_in_python'},
 {'sim': 0.76776969, 'skill': u'python__+_hive_profile'},
 {'sim': 0.7677325, 'skill': u'authentication_auth'},
 {'sim': 0.76773226, 'skill': u'powerbuilder_115'}]

In [25]:
get_top('ec2')

[{'sim': 0.99999994, 'skill': u'ec2'},
 {'sim': 0.88580132, 'skill': u'route53__aws'},
 {'sim': 0.84055293, 'skill': u'linux__systems'},
 {'sim': 0.80940634, 'skill': u'elastic_search_service'},
 {'sim': 0.78297925, 'skill': u'elasticache'},
 {'sim': 0.77151394, 'skill': u's3'},
 {'sim': 0.75670588, 'skill': u'aws_server_management'},
 {'sim': 0.74875033, 'skill': u'task_automation'},
 {'sim': 0.74709171, 'skill': u'python__aws'},
 {'sim': 0.72643453, 'skill': u'api_gateway_service'}]

In [62]:
get_top('hadoop')

[{'sim': 1.0, 'skill': u'hadoop'},
 {'sim': 0.82934582, 'skill': u'azureaws_paas'},
 {'sim': 0.82934582, 'skill': u'bigdate'},
 {'sim': 0.82934582, 'skill': u'spark_storm'},
 {'sim': 0.82922697, 'skill': u'microstrategy_100'},
 {'sim': 0.82921052, 'skill': u'strong_java_cod'},
 {'sim': 0.82901168, 'skill': u'machine_learn_analytics_tool'},
 {'sim': 0.82820946, 'skill': u'seek_professionals'},
 {'sim': 0.82820946, 'skill': u'well_verse'},
 {'sim': 0.82809293,
  'skill': u'technical_skills__sku__diversify__diversify__sr_track_lead'}]

In [ ]:
import time
import scipy
k=1000

start_time = time.time()
U, S, Vt = scipy.sparse.linalg.svds(matrix_tfidf_csc, k=k, return_singular_vectors='u')
print "k = ", k, "SVD Time:", time.time() - start_time

In [14]:
import cPickle as pickle
pickle.dump( U, open( "U_1000.p", "wb" ) )
pickle.dump( S, open( "S_1000.p", "wb" ) )
pickle.dump( Vt, open( "Vt_1000.p", "wb" ) )

In [2]:
import cPickle as pickle
U = pickle.load( open( "U_1000.p", "rb" ) )
S = pickle.load( open( "S_1000.p", "rb" ) )
# Vt = pickle.load( open( "Vt_1000.p", "rb" ) )

In [3]:
U.shape, S.shape

((113256, 1000), (1000,))

In [5]:
from scipy import linalg
SS = linalg.diagsvd(S, 1000, 1000)

In [6]:
lsa_vector = U.dot(SS)

In [53]:
pickle.dump( lsa_vector, open( "lsa_vector.p", "wb" ) )

In [75]:
def get_lsa_vector(t):
    try:
        return lsa_vector[dictionary.token2id[skill_transform.skill_transform(t)]]
    except:
        return None
def id2skill(id):
    return dictionary[id]

In [19]:
get_lsa_vector('html5')

array([  1.70548343e-02,  -8.72186041e-03,  -7.04514842e-03,
         4.25405783e-03,  -1.42670367e-02,   2.03652809e-02,
        -1.03807771e-02,   2.72669581e-03,  -2.33811088e-02,
         7.98356176e-03,   2.77489491e-02,  -6.73097154e-03,
        -1.50858934e-02,   3.13137796e-03,  -1.29944356e-02,
         2.89428239e-03,   5.76469312e-03,  -3.46019364e-03,
         2.51347622e-03,  -1.68941370e-03,  -5.13032215e-04,
        -2.93572852e-03,  -2.96446072e-02,  -4.41323867e-02,
         6.90339678e-03,  -6.06397430e-03,   5.21747732e-03,
         1.24877082e-02,   3.77449275e-02,   1.97130865e-02,
        -3.79984711e-02,  -1.62417470e-02,   2.10171101e-02,
         2.11145981e-02,   3.92107759e-02,  -5.53899506e-02,
        -1.88821920e-02,  -8.42112010e-02,   1.36607798e-02,
        -2.49554608e-02,  -2.08391521e-02,  -5.09009504e-02,
        -1.60664083e-02,   1.40778021e-02,   1.33907616e-02,
        -4.51900780e-02,   1.26087150e-02,  -1.74892233e-01,
        -1.69918978e-05,

In [54]:
from scipy.spatial.distance import cosine
1 - cosine(get_lsa_vector('JavaScript'), get_lsa_vector('angularjs'))

0.33249005850346802

In [55]:
lsa_copus = gensim.matutils.Dense2Corpus(lsa_vector.T)
lsa_sim = similarities.MatrixSimilarity(lsa_copus)

In [56]:
lsa_sim = similarities.MatrixSimilarity(lsa_copus)

In [57]:
lsa_sim.save('lsa_sin.index')

In [90]:
def get_top(skill, n = 10):
    sim_list = lsa_sim[get_lsa_vector(skill)]
    top_index = sim_list.argsort()[-n:][::-1]
    res = []
    for i in top_index:
        res.append({ 'skill': id2skill(i), 'sim': sim_list[i] })
        
    return res

get_top('react.js')

[{'sim': 1.0, 'skill': u'react'},
 {'sim': 0.90243566, 'skill': u'typescript_net'},
 {'sim': 0.90222073, 'skill': u'java_restful_webservice'},
 {'sim': 0.90178955, 'skill': u'full_stack_developers_node'},
 {'sim': 0.901573, 'skill': u'php_use_javascript'},
 {'sim': 0.89905322, 'skill': u'or_vuejs'},
 {'sim': 0.89905322, 'skill': u'es6_experience'},
 {'sim': 0.89780712, 'skill': u'flux_be_strongly_need'},
 {'sim': 0.87958473, 'skill': u'react_native__javascript'},
 {'sim': 0.87068141, 'skill': u'meteor_low_level_publish_api'}]

In [91]:
get_top('html')

[{'sim': 1.0, 'skill': u'html'},
 {'sim': 0.68476021, 'skill': u'css__suitecommerce'},
 {'sim': 0.61157656, 'skill': u'ui_developer_ui'},
 {'sim': 0.61113369, 'skill': u'mvc_pattern_etc'},
 {'sim': 0.61053967, 'skill': u'"rest_web_service"'},
 {'sim': 0.60980034, 'skill': u'es2015'},
 {'sim': 0.60957491, 'skill': u'adobe_campaign_neolane'},
 {'sim': 0.60945791, 'skill': u'non_relational'},
 {'sim': 0.60848337, 'skill': u'frond_end_developers'},
 {'sim': 0.60848337, 'skill': u'jsp_backbone'}]

In [92]:
get_top('jsp')

[{'sim': 1.0, 'skill': u'jsp'},
 {'sim': 0.91569203, 'skill': u'websphereportalserver'},
 {'sim': 0.86018842, 'skill': u'ejb30'},
 {'sim': 0.85698497, 'skill': u'custom_tag'},
 {'sim': 0.84459019, 'skill': u'integration_with_filenet'},
 {'sim': 0.82186908, 'skill': u'pega_ssa_cpm'},
 {'sim': 0.82186908, 'skill': u'decision_managemen_servlets'},
 {'sim': 0.79775, 'skill': u'jss'},
 {'sim': 0.79299128, 'skill': u'hibernate__j2ee'},
 {'sim': 0.77587926, 'skill': u'mc_captiva_inputaccel'}]

In [96]:
get_top('xml')

[{'sim': 1.0, 'skill': u'xml'},
 {'sim': 0.84527904, 'skill': u'eqc'},
 {'sim': 0.84527904, 'skill': u'electronic_quality_control'},
 {'sim': 0.8443799, 'skill': u'xml_technical'},
 {'sim': 0.8443799, 'skill': u'xml_consultant'},
 {'sim': 0.84422278, 'skill': u'otm_database'},
 {'sim': 0.84422278, 'skill': u'oracle__scm'},
 {'sim': 0.84422082, 'skill': u'integration_data_flow'},
 {'sim': 0.83976465, 'skill': u'strong_c#_skills'},
 {'sim': 0.83545542, 'skill': u'web_b'}]

In [97]:
get_top('web')

[{'sim': 0.99999994, 'skill': u'web'},
 {'sim': 0.92705125, 'skill': u'php_theme'},
 {'sim': 0.92705125, 'skill': u'magento_theme'},
 {'sim': 0.92705125, 'skill': u'oscommerce_theme'},
 {'sim': 0.9269619, 'skill': u'ui_developers_/_web'},
 {'sim': 0.9269619, 'skill': u'graphics_designer/multimedia/web_developer'},
 {'sim': 0.92693973, 'skill': u'country_specific'},
 {'sim': 0.92693162, 'skill': u'responsive_website_designer'},
 {'sim': 0.92693162, 'skill': u'web_sphere_integration_developer'},
 {'sim': 0.92693162, 'skill': u'websphere_esb_developer'}]

In [98]:
get_top('php')

[{'sim': 1.0, 'skill': u'php'},
 {'sim': 0.87152123, 'skill': u'magento_lead_developer'},
 {'sim': 0.86700761, 'skill': u'handson_team_management'},
 {'sim': 0.80870068, 'skill': u'codeigneter'},
 {'sim': 0.80158424, 'skill': u'php_consultant'},
 {'sim': 0.799101, 'skill': u'support_role'},
 {'sim': 0.79307067, 'skill': u'mysq'},
 {'sim': 0.77547383, 'skill': u'mysql_syntax'},
 {'sim': 0.77547288, 'skill': u'oops_php'},
 {'sim': 0.77257746, 'skill': u'mysqli'}]

In [99]:
get_top('mysql')

[{'sim': 0.99999994, 'skill': u'mysql'},
 {'sim': 0.8915056, 'skill': u'c#_experience_a_plus'},
 {'sim': 0.86419296, 'skill': u'magento_2x'},
 {'sim': 0.7698729, 'skill': u'java__mysql'},
 {'sim': 0.7698729, 'skill': u'restful__api'},
 {'sim': 0.76838398, 'skill': u'neo4j_graph_db'},
 {'sim': 0.7551164, 'skill': u'chinese_speak'},
 {'sim': 0.73726428, 'skill': u'scalability__java'},
 {'sim': 0.69180024, 'skill': u'agile__php'},
 {'sim': 0.65530908, 'skill': u'my_sql_server_developer'}]

In [100]:
get_top('neo4j')

[{'sim': 0.99999994, 'skill': u'neo4j'},
 {'sim': 0.66968024, 'skill': u'sql_store'},
 {'sim': 0.65675211, 'skill': u'cassandra'},
 {'sim': 0.64054978, 'skill': u'informix_relational_platform'},
 {'sim': 0.64054978, 'skill': u'vormetric_encryption'},
 {'sim': 0.64054978, 'skill': u'informix_ids_117'},
 {'sim': 0.63360018, 'skill': u'couchdb'},
 {'sim': 0.61383736, 'skill': u'etl_pipe_line'},
 {'sim': 0.61383736, 'skill': u'resdshift'},
 {'sim': 0.60721052, 'skill': u'postgresql__cassandra'}]

In [102]:
get_top('python')

[{'sim': 1.0, 'skill': u'python'},
 {'sim': 0.82759118, 'skill': u'stackless'},
 {'sim': 0.82758868, 'skill': u'such_as_sas'},
 {'sim': 0.82758868, 'skill': u'and_spss'},
 {'sim': 0.82758713, 'skill': u'molecular_simulation'},
 {'sim': 0.82758713, 'skill': u'computational_chemist'},
 {'sim': 0.82758713, 'skill': u'quantum_mechanics'},
 {'sim': 0.82758713, 'skill': u'quantum_chemistry'},
 {'sim': 0.82755947, 'skill': u'molecular_model'},
 {'sim': 0.82744694, 'skill': u'statistical_background'}]

In [103]:
get_top('ruby')

[{'sim': 1.0, 'skill': u'ruby'},
 {'sim': 0.89971209, 'skill': u'exp_with_linux'},
 {'sim': 0.89914733, 'skill': u'dev_ops_engineer'},
 {'sim': 0.89613152, 'skill': u'redhat_java'},
 {'sim': 0.89596373, 'skill': u'or_javascript_be_ideal'},
 {'sim': 0.8555907, 'skill': u'chef_server'},
 {'sim': 0.83739722, 'skill': u'aws__automation_test_plan'},
 {'sim': 0.83739722, 'skill': u'automation_test_plan'},
 {'sim': 0.80441195, 'skill': u'sql_map'},
 {'sim': 0.80441195, 'skill': u'and_kali_linux'}]

In [9]:
import time
import gensim 
from gensim import corpora, models, similarities

start_time = time.time()
dictionary = corpora.Dictionary(dice_data_train)
dictionary.save('dice_data_train.dict')

dice_data_train_number = [dictionary.doc2bow(t) for t in dice_data_train]
corpora.MmCorpus.serialize('dice_data_train_number.mm', dice_data_train_number) # store to disk

# TfidfModel
tfidf = models.TfidfModel(dice_data_train_number)

# New copus TFIDF
corpus_tfidf = tfidf[dice_data_train_number]
tfidf.save('thesis_tfidf.model')

print 'TF-IDF Time: ', time.time() - start_time
start_time = time.time()

# Convert tp Sparse CSC44053
matrix_tfidf_csc = gensim.matutils.corpus2csc(corpus_tfidf, num_terms=len(dictionary))

end_time = time.time()
print "Convert to Spare Matrix:", time.time() - start_time

2017-06-13 13:04:56,555 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-06-13 13:04:57,047 : INFO : adding document #10000 to Dictionary(11367 unique tokens: [u'capital_call', u'biopython', u'supply_side_platform', u'umts', u'white_box']...)
2017-06-13 13:04:57,303 : INFO : adding document #20000 to Dictionary(21829 unique tokens: [u'chennal_sales', u'capital_call', u'biopython', u'liquidity', u'retail_assistant_manager']...)
2017-06-13 13:04:57,557 : INFO : adding document #30000 to Dictionary(30690 unique tokens: [u'cng_station', u'healthcare_advertise', u'head_cashier', u'white_box', u'outstanding_problem_solve']...)
2017-06-13 13:04:57,829 : INFO : adding document #40000 to Dictionary(38672 unique tokens: [u'cng_station', u'road_project', u'mandrill_act', u'healthcare_advertise', u'head_cashier']...)
2017-06-13 13:04:58,101 : INFO : adding document #50000 to Dictionary(46099 unique tokens: [u'cng_station', u'road_project', u'mandrill_act', u'colonoscopy', u'heal

TF-IDF Time:  131.076937199
Convert to Spare Matrix: 28.3946559429


In [118]:
matrix_tfidf_csc.shape

(113256, 1438906)

In [124]:
import scipy

start_time = time.time()
U, S, Vt = scipy.sparse.linalg.svds(matrix_tfidf_csc, k=min(matrix_tfidf_csc.shape) - 1)
print "SVD Time:", time.time() - start_time

MemoryError: 

In [122]:
min(matrix_tfidf_csc.shape)

113256

In [59]:
tfidf = models.TfidfModel(dice_data_train_number)

2017-06-11 21:22:45,770 : INFO : collecting document frequencies
2017-06-11 21:22:45,771 : INFO : PROGRESS: processing document #0
2017-06-11 21:22:45,883 : INFO : PROGRESS: processing document #10000
2017-06-11 21:22:45,927 : INFO : PROGRESS: processing document #20000
2017-06-11 21:22:45,968 : INFO : PROGRESS: processing document #30000
2017-06-11 21:22:46,013 : INFO : PROGRESS: processing document #40000
2017-06-11 21:22:46,055 : INFO : PROGRESS: processing document #50000
2017-06-11 21:22:46,095 : INFO : PROGRESS: processing document #60000
2017-06-11 21:22:46,137 : INFO : PROGRESS: processing document #70000
2017-06-11 21:22:46,180 : INFO : PROGRESS: processing document #80000
2017-06-11 21:22:46,225 : INFO : PROGRESS: processing document #90000
2017-06-11 21:22:46,268 : INFO : PROGRESS: processing document #100000
2017-06-11 21:22:46,309 : INFO : PROGRESS: processing document #110000
2017-06-11 21:22:46,350 : INFO : PROGRESS: processing document #120000
2017-06-11 21:22:46,390 : 

In [60]:
corpus_tfidf = tfidf[dice_data_train_number]

In [76]:
tfidf.save('thesis_tfidf.model')

2017-06-12 09:22:49,229 : INFO : saving TfidfModel object under thesis_tfidf.model, separately None
2017-06-12 09:22:49,342 : INFO : saved thesis_tfidf.model


In [75]:
import gensim
SVD = gensim.models.lsimodel.stochastic_svd(corpus_tfidf, rank=300, num_terms=len(dictionary))

2017-06-12 09:19:51,464 : INFO : using 300 extra samples and 0 power iterations
2017-06-12 09:19:51,465 : INFO : 1st phase: constructing (113256, 600) action matrix
2017-06-12 09:19:52,038 : INFO : PROGRESS: at document #0
2017-06-12 09:19:53,408 : INFO : PROGRESS: at document #20000
2017-06-12 09:19:54,648 : INFO : PROGRESS: at document #40000
2017-06-12 09:19:55,922 : INFO : PROGRESS: at document #60000
2017-06-12 09:19:57,167 : INFO : PROGRESS: at document #80000
2017-06-12 09:19:58,405 : INFO : PROGRESS: at document #100000
2017-06-12 09:19:59,633 : INFO : PROGRESS: at document #120000
2017-06-12 09:20:00,851 : INFO : PROGRESS: at document #140000
2017-06-12 09:20:02,074 : INFO : PROGRESS: at document #160000
2017-06-12 09:20:03,285 : INFO : PROGRESS: at document #180000
2017-06-12 09:20:04,469 : INFO : PROGRESS: at document #200000
2017-06-12 09:20:05,601 : INFO : PROGRESS: at document #220000
2017-06-12 09:20:06,757 : INFO : PROGRESS: at document #240000
2017-06-12 09:20:07,925 :

In [89]:
corpus_tfidf.save('tfidf.txt')

/usr/lib64/python2.7/site-packages/gensim/interfaces.py:60: UserWarning: corpus.save() stores only the (tiny) iteration object; to serialize the actual corpus content, use e.g. MmCorpus.serialize(corpus)
  warnings.warn("corpus.save() stores only the (tiny) iteration object; "
2017-06-12 09:32:56,610 : INFO : saving TransformedCorpus object under tfidf.txt, separately None
2017-06-12 09:33:24,188 : INFO : saved tfidf.txt


In [111]:
dir(corpus_tfidf)
matrix_tfidf_csc = gensim.matutils.corpus2csc(corpus_tfidf, num_terms=len(dictionary))

In [112]:
matrix_tfidf_csc.shape

(113256, 1438906)

In [110]:
html5 = matrix_tfidf_csc[dictionary.token2id['html5']].todense()
css3 = matrix_tfidf_csc[dictionary.token2id['css3']].todense()

1 - spatial.distance.cosine(html5, css3)

0.5152121594361031

In [115]:
matrix_tfidf_csc[113255]

<1x1438906 sparse matrix of type '<type 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Column format>